In [10]:
import numpy as np
import pandas as pd
import sklearn
from PIL import Image
import os

In [11]:
images_df = pd.read_csv('complete_data.csv')
images_df.describe()

,Unnamed: 0,Breslow,age_approx,id1,num
count,150.000000,10.000000,80.000000,150.000000,150.000000
mean,74.500000,0.830000,43.000000,75.853333,74.500000
std,43.445368,0.819621,16.699125,43.859505,43.445368
min,0.000000,0.250000,15.000000,1.000000,0.000000
25%,37.250000,0.500000,30.000000,38.250000,37.250000
50%,74.500000,0.575000,40.000000,75.500000,74.500000
75%,111.750000,0.775000,55.000000,113.750000,111.750000
max,149.000000,3.100000,85.000000,153.000000,149.000000


In [12]:
# Clean up data

images_df = images_df.drop(['Unnamed: 0', 'id1', 'site'], axis=1)
images_df['Clark'] = images_df['Clark'].map({'I': 1, 'II': 2, 'III': 3}).fillna(0).astype(int)
images_df['Breslow'] = images_df['Breslow'].fillna(0)
images_df['diagnosis_confirm_type'] = images_df['diagnosis_confirm_type'].map({'histopathology': 1}).fillna(0).astype(int)
images_df['melanocytic'] = images_df['melanocytic'].map({True: 1, False: 0})
images_df['benign_malignant'] = images_df['benign_malignant'].map({'benign': 1, 'malignant': 0}).astype(int)
images_df['age_approx'] = images_df['age_approx'].fillna(images_df['age_approx'].dropna().mean())

images_df.head()

,Breslow,Clark,age_approx,anatom_site_general,benign_malignant,diagnosis,diagnosis_confirm_type,localization,melanocytic,num,sex
0,0.0,0,55.0,anterior torso,1,dysplastic nevus,0,Abdomen,1,0,female
1,0.0,0,30.0,anterior torso,1,dysplastic nevus,0,Abdomen,1,1,female
2,0.0,0,60.0,upper extremity,0,Melanoma in situ,1,Upper limb,1,2,female
3,0.0,0,30.0,upper extremity,1,dysplstic nevus,0,Upper limb,1,3,male
4,0.0,0,80.0,posterior torso,0,Melanoma,1,Back,1,4,male


In [13]:
images_df = pd.get_dummies(images_df)

images_df.head()

,Breslow,Clark,age_approx,benign_malignant,diagnosis_confirm_type,num,anatom_site_general_anterior torso,anatom_site_general_lower extremity,anatom_site_general_posterior torso,anatom_site_general_upper extremity,...,diagnosis_nevus nos,localization_Abdomen,localization_Back,localization_Breast,localization_Pubis,localization_Upper limb,localization_lower limb,melanocytic_1,sex_female,sex_male
0,0.0,0,55.0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,1,0
1,0.0,0,30.0,1,0,1,1,0,0,0,...,0,1,0,0,0,0,0,1,1,0
2,0.0,0,60.0,0,1,2,0,0,0,1,...,0,0,0,0,0,1,0,1,1,0
3,0.0,0,30.0,1,0,3,0,0,0,1,...,0,0,0,0,0,1,0,1,0,1
4,0.0,0,80.0,0,1,4,0,0,1,0,...,0,0,1,0,0,0,0,1,0,1


In [14]:
# Now, we want to actual images

def get_ith_image(i):
    I = np.array(Image.open('img/image%03d.jpg' % i).convert('RGB'))
    I_red = I[:,:,0].flatten()
    I_green = I[:,:,1].flatten()
    I_blue = I[:,:,2].flatten()
    feature_vector = np.hstack((I_red, I_green, I_blue))
    return feature_vector
    
num_images = len(os.listdir('img'))
s = pd.DataFrame(np.matrix([get_ith_image(i) for i in range(num_images)]))
s.shape

(150, 840000)

In [15]:
image_final_df = pd.concat((images_df, s), axis='columns')
image_final_df.head()

,Breslow,Clark,age_approx,benign_malignant,diagnosis_confirm_type,num,anatom_site_general_anterior torso,anatom_site_general_lower extremity,anatom_site_general_posterior torso,anatom_site_general_upper extremity,...,839990,839991,839992,839993,839994,839995,839996,839997,839998,839999
0,0.0,0,55.0,1,0,0,1,0,0,0,...,255,255,255,255,255,255,254,255,239,255
1,0.0,0,30.0,1,0,1,1,0,0,0,...,218,220,220,223,223,223,225,214,206,251
2,0.0,0,60.0,0,1,2,0,0,0,1,...,161,158,156,154,151,149,150,150,106,227
3,0.0,0,30.0,1,0,3,0,0,0,1,...,236,237,238,239,240,240,233,239,228,255
4,0.0,0,80.0,0,1,4,0,0,1,0,...,126,126,126,126,126,126,126,137,103,232


In [16]:
# Let's train a model!

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC

X = image_final_df.drop(['benign_malignant'], axis=1)
y = image_final_df['benign_malignant']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y)

clf = SVC(random_state=0).fit(X_train, y_train)

training_error = clf.score(X_train, y_train)
test_error = clf.score(X_test, y_test)

print 'Training Error: %f' % (1 - training_error)
print 'Test Error: %f' % (1 - test_error)

Training Error: 0.000000
Test Error: 0.220000
